This notebook  illustrates how you can use the langchain with custom pdf data & chat it using autogen agent

In [ ]:
%pip install pyautogen~=0.1.0  langchain openai tiktoken lancedb pypdf -q -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.8/811.8 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.9/111.9 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.2/295.2 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.6/21.6 MB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain_community.vectorstores import LanceDB

Requirements
AutoGen requires Python>=3.8. To run this notebook example, please install pyautogen:


In [ ]:
!pip install pyautogen

In [ ]:
import lancedb

embeddings = OpenAIEmbeddings(openai_api_key="sk-yourapikey")

In [ ]:
# !wget -O uniswap_v3.pdf https://uniswap.org/whitepaper-v3.pdf
!wget -O food.pdf https://pdf.usaid.gov/pdf_docs/PA00TBCT.pdf

--2024-02-11 04:40:16--  https://pdf.usaid.gov/pdf_docs/PA00TBCT.pdf
Resolving pdf.usaid.gov (pdf.usaid.gov)... 23.7.61.67, 2600:1408:ec00:380::1923, 2600:1408:ec00:38f::1923
Connecting to pdf.usaid.gov (pdf.usaid.gov)|23.7.61.67|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6419525 (6.1M) [application/pdf]
Saving to: ‘food.pdf’

food.pdf            100%[===================>]   6.12M  --.-KB/s    in 0.1s    

2024-02-11 04:40:16 (42.7 MB/s) - ‘food.pdf’ saved [6419525/6419525]



create OAI_CONFIG_LIST.json file in pwd & upload
in it


[
   {
     "model": "gpt-4",
     "api_key": "sk-yourapikey"
   }
]

In [ ]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST.json",
    filter_dict={
        "model": ["gpt-4"],
    },
)

# create file name with OAI_CONFIG_LIT.json & put below authentications code

In [ ]:
# create file name with OAI_CONFIG_LIT.
[{"model": "gpt-4", "api_key": "sk-yourapikey"}]

In [ ]:
loaders = [PyPDFLoader("./food.pdf")]
docs = []
for l in loaders:
    docs.extend(l.load())
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(docs)

In [ ]:
import lancedb

embeddings = OpenAIEmbeddings(openai_api_key="sk-yourapikey")

db = lancedb.connect("/tmp/lancedb")
table = db.create_table(
    "my_table",
    data=[
        {
            "vector": embeddings.embed_query("Hello food"),
            "text": "Hello food",
            "id": "1",
        }
    ],
    mode="overwrite",
)

vectorstore = LanceDB.from_documents(docs, embeddings, connection=table)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [ ]:
qa = ConversationalRetrievalChain.from_llm(
    OpenAI(
        temperature=0,
        openai_api_key="sk-yourapikey",
    ),
    vectorstore.as_retriever(),
    memory=ConversationBufferMemory(memory_key="chat_history", return_messages=True),
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [ ]:
def answer_food_question(question):
    response = qa({"question": question})
    return response["answer"]

In [ ]:
question = "what is good food"
answer_food_question(question)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


' Good food is food that provides the recommended amounts of nutrients for the body to perform all its physiological activities. It is important to eat the right food, at the right time, in the right amounts, and prepared correctly in order to maintain a balanced diet and promote good nutrition. Good food is essential for physical and cognitive development and can improve overall health and quality of life.'

In [ ]:
!pip install pyautogen

#### 4. Set up AutoGen user agent and assistant agent with function calling enabled.

In [ ]:
llm_config = {
    "request_timeout": 600,
    "seed": 42,
    "config_list": config_list,
    "temperature": 0,
    "functions": [
        {
            "name": "answer_food_question",
            "description": "Answer to any quetion related to food & provided answer_food_question ",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {
                        "type": "string",
                        "description": "The question to ask in relation to food",
                    }
                },
                "required": ["question"],
            },
        }
    ],
}

In [ ]:
# create an AssistantAgent instance named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config,
)
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    code_execution_config={"work_dir": "."},
    llm_config=llm_config,
    system_message="""Reply TERMINATE if the task has been solved at full satisfaction.
Otherwise, reply CONTINUE, or the reason why the task is not solved yet.""",
    function_map={"answer_food_question": answer_food_question},
)

In [ ]:
# the assistant receives a message from the user, which contains the task description
user_proxy.initiate_chat(
    assistant,
    message="""
what is good food?
""",
)

user_proxy (to assistant):


what is good food?


--------------------------------------------------------------------------------
assistant (to user_proxy):

***** Suggested function Call: answer_food_question *****
Arguments: 

{
  "question": "what is good food?"
}
*********************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION answer_food_question...
user_proxy (to assistant):

***** Response from calling function "answer_food_question" *****
 Good food is food that is able to provide the recommended amounts of nutrients for the body to perform all its physiological activities. It is important for our health and well-being because it helps us maintain a balanced diet, promotes physical and cognitive development, and protects us from foodborne illnesses. Good food also ensures that we have enough energy for physical activity and basic body functions, and it helps us maintain a he

In [ ]:
# the assistant receives a message from the user, which contains the task description
user_proxy.initiate_chat(
    assistant,
    message="""
please explain me essential minerals, sources, functions and symptoms of
deficiency?
""",
)

user_proxy (to assistant):


please explain me essential minerals, sources, functions and symptoms of
deficiency?


--------------------------------------------------------------------------------
assistant (to user_proxy):

Sure, here are some essential minerals, their sources, functions, and symptoms of deficiency:

1. Calcium:
   - Sources: Dairy products, leafy green vegetables, fish with edible bones (like sardines and salmon), fortified foods.
   - Functions: Necessary for bone health, muscle function, nerve transmission, blood clotting.
   - Deficiency Symptoms: Osteoporosis, rickets in children, muscle cramps, dental problems.

2. Iron:
   - Sources: Red meat, poultry, eggs, fruits, green vegetables, fortified bread.
   - Functions: Essential for the production of red blood cells, helps in oxygen transport.
   - Deficiency Symptoms: Anemia, fatigue, weakness, immune system problems.

3. Magnesium:
   - Sources: Nuts, seeds, whole grains, green leafy vegetables, fish, beans, yog

In [ ]:
# the assistant receives a message from the user, which contains the task description
user_proxy.initiate_chat(
    assistant,
    message="""
which food Keeps eyes healthy ?
""",
)

user_proxy (to assistant):


which food Keeps eyes healthy ?


--------------------------------------------------------------------------------
assistant (to user_proxy):

***** Suggested function Call: answer_food_question *****
Arguments: 

{
  "question": "which food Keeps eyes healthy ?"
}
*********************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION answer_food_question...
user_proxy (to assistant):

***** Response from calling function "answer_food_question" *****
 Foods that are rich in Vitamin A, such as yellow/orange fruits and vegetables, dark green and deep yellow fruits and vegetables, liver, egg yolk, dairy products, and margarine can help maintain healthy eyes.
*****************************************************************

--------------------------------------------------------------------------------
assistant (to user_proxy):

Foods that are rich in Vitamin A 